In [736]:
%matplotlib inline

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import psycopg2
import textatistic
import seaborn as sbn
from altair import Chart, X, Y, Color, Scale
import altair as alt
from vega_datasets import data
import requests
from bs4 import BeautifulSoup
matplotlib.style.use('ggplot')
# for plotly py.offline.init_notebook_mode()

### Loading the data

In [737]:
YEAR = 2012

In [738]:
# delete
with open('/Users/bradleymiller/Runestone/Research/factbook/{}/docs/notesanddefs.html'.format(YEAR)) as wff:
    field_list = BeautifulSoup(wff.read())
    

In [739]:
fl = field_list.select('div[name]')

In [740]:
import re
if YEAR < 2015:
    cfield = 'table'
else:
    cfield = 'div'
    
fl = field_list.find_all(cfield, id=re.compile("^\d+$") )

In [741]:
fl[0].select('td')[0].text

'Administrative divisions'

In [742]:
len(fl)

173

In [743]:
for f in fl:
    print(f['id'], f.select('td')[0].text)
    

2051 Administrative divisions
2010 Age structure
2052 Agriculture - products
2053 Airports
2030 Airports - with paved runways
2031 Airports - with unpaved runways
2147 Area
2023 Area - comparative
2028 Background
2054 Birth rate
2213 Broadcast media
2056 Budget
2222 Budget surplus (+) or deficit (-)
2057 Capital
2254 Carbon dioxide emissions from consumption of energy
2207 Central bank discount rate
2224 Children under the age of 5 years underweight
2059 Climate
2060 Coastline
2208 Commercial bank prime lending rate
2138 Communications - note
2063 Constitution
2142 Country name
2242 Crude oil - exports
2243 Crude oil - imports
2241 Crude oil - production
2244 Crude oil - proved reserves
2187 Current account balance
2066 Death rate
2079 Debt - external
2257 Demographic profile
2006 Dependency status
2068 Dependent areas
2007 Diplomatic representation from the US
2149 Diplomatic representation in the US
2070 Disputes - international
2172 Distribution of family income - Gini index
2216 Dr

In [744]:
required_columns = [
'Population',
'Area', 
'Coastline', 
'Climate',
'Net migration',
'Birth rate',
'Death rate',
'Infant mortality rate',
'Literacy',
'GDP (purchasing power parity)',
'Government type',
'Inflation rate',
'Health expenditures',
'GDP - composition, by sector of origin',
'Internet users'
]

In [766]:
data = {}
if YEAR < 2015:  # this still needs some work!!
    outer_table = 'table#fieldlisting'
    inner_cell  = 'td.category_data'
    inner_table = 'table[id]'
else:
    outer_table = 'table#fieldListing'
    inner_cell = 'td.fieldData'
    inner_table = 'tr'
for f in fl:
    field_name = f.select_one('td').text
    if field_name in required_columns:
        data[field_name] = {}
        with open('/Users/bradleymiller/Runestone/Research/factbook/{}/fields/{}.html'.format(YEAR, f['id'])) as xf:
            soup = BeautifulSoup(xf.read())
            if YEAR > 2015:
                cl = soup.select_one(outer_table).select(inner_table)
                for country in cl[1:]:
                    two_letter = country['id']
                    data[field_name][two_letter] = country.select_one(inner_cell).text.strip()
            else:
                cl = soup.select('table[id]')
                for country in cl[4:]:
                    two_letter = country.select_one('td a[name]')['name']
                    data[field_name][two_letter] = country.select_one('td.category_data').text.strip()
                
            

In [746]:
text = open('/Users/bradleymiller/Runestone/Research/factbook/{}/fields/{}.html'.format(YEAR, '2154')).read()


In [747]:
soup = BeautifulSoup(text)

In [748]:
cl = soup.select_one(outer_table).select(inner_table)



In [749]:
data['Country'] = {}
for country in cl[1:]:
    two_letter = country['id']
    data['Country'][two_letter] = country.select_one('td a').text.strip()

    

In [750]:
facts = pd.DataFrame(data)

In [751]:
facts['Area'] = facts.Area.str.replace('\s[mb]illion','000000').str.extract('total: ?([0-9,.]+) sq km')[0].str.replace(',','').astype(float)
#facts.Area.str.extract('total: ?([0-9,.]+) sq km')

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [ ]:
facts['Birth rate'] = facts['Birth rate'].str.extract('([0-9.,]+) births.*').astype(float)

In [ ]:
facts['Coastline'] = facts.Coastline.str.extract('([0-9,.]+)')
facts['Coastline'] = facts.Coastline.str.replace(',','').astype(float)

In [ ]:
facts.head()

In [ ]:
facts['Death rate'] = facts['Death rate'].str.extract('([0-9.,]+) deaths.*').astype(float)

In [ ]:
facts['GDP'] = facts['GDP (purchasing power parity)'].str.extract('\$([0-9.,]+ [bmtr]+illion).*')


In [ ]:
facts['GDP'] = facts.GDP.str.replace(' million','e6')
facts['GDP'] = facts.GDP.str.replace(' billion','e9')
facts['GDP'] = facts.GDP.str.replace(' trillion','e12')      

In [ ]:
facts['GDP'] = facts.GDP.astype(float)

In [ ]:
facts.drop('GDP (purchasing power parity)',axis=1, inplace=True)

Let's convert health expenditures to a real $ now if we want to know percentages we can always convert back later

In [ ]:
facts['Health expenditures'] = facts['Health expenditures'].str.extract('([0-9.]+)% of GDP').astype(float) * 0.01


In [ ]:
facts['Health expenditures'] = facts['Health expenditures'] * facts.GDP.astype(float)

In [ ]:
facts.head()

In [ ]:
facts['Internet users'] = facts['Internet users'].str.extract('total: ([0-9,]+)')
facts['Internet users'] = facts['Internet users'].str.replace(',','').astype(float)

In [ ]:
facts['GDP - composition, by sector of origin'] = facts['GDP - composition, by sector of origin'].str.replace('\n',' ')
gdp_split = facts['GDP - composition, by sector of origin'].str.extract('agriculture:\s(?P<agriculture>[0-9.]+)%.*industry:\s(?P<industry>[0-9.]+)%.*services:\s(?P<service>[0-9.]+)%.*')
facts = facts.merge(gdp_split, left_index=True, right_index=True)
facts.drop('GDP - composition, by sector of origin', axis=1, inplace=True)
facts.head()

In [ ]:
facts['Literacy'] = facts.Literacy.str.extract('.*population:\s([0-9.]+)%').astype(float)


In [ ]:
facts['Population'] = facts.Population.str.extract('([0-9,.]+)')
facts['Population'] = facts.Population.str.replace(',','')
facts['Population'] = facts.Population.map(lambda x: 'NaN' if x == '' else x).astype(float)

In [ ]:
facts['Infant mortality rate'] = facts['Infant mortality rate'].str.extract('total: ([0-9.]+)').astype(float)

In [ ]:
facts['Year'] = YEAR

In [ ]:
facts.to_csv('factbook{}.csv'.format(YEAR), index_label='Code2')